In [1]:
import pandas as pd                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Reading Dataset

In [2]:
df = pd.read_csv('training_data.csv')
df.head(5)

,BUS ID,BR_Repair status,Problem,SPN_247 (Engine Total Hours of Operation),SPN_190 (Engine Speed),SPN_513 (Actual Engine - Percent Torque),SPN_250 (Engine Total Fuel Used),SPN_110 (Engine Coolant Temperature),SPN_183(Engine Fuel Rate),SPN_184 (Engine Instantaneous Fuel Economy),SPN_92\t(Engine Percent Load At Current Speed),Grade
0,VG8671,Major,Engine,12267,721,130,10468,79,1756,128,255,B
1,VG8630,MINOR,Engine,35745,755,130,110137,80,2082,128,255,C
2,VAF5480,MINOR,Engine,299,801,26,2179,76,1995,6,33,B
3,VAF5489,MINOR,Engine,247,808,20,1991,80,1761,20,25,B
4,VAF7134,MINOR,Engine,14099,639,18,111666,78,1971,24,22,B


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 12 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   BUS ID                                         48 non-null     object
 1   BR_Repair status                               48 non-null     object
 2   Problem                                        48 non-null     object
 3   SPN_247 (Engine Total Hours of Operation)      48 non-null     int64 
 4   SPN_190 (Engine Speed)                         48 non-null     int64 
 5   SPN_513 (Actual Engine - Percent Torque)       48 non-null     int64 
 6   SPN_250 (Engine Total Fuel Used)               48 non-null     int64 
 7   SPN_110 (Engine Coolant Temperature)           48 non-null     int64 
 8   SPN_183(Engine Fuel Rate)                      48 non-null     int64 
 9   SPN_184 (Engine Instantaneous Fuel Economy)    48 non-null     int6

In [4]:

# Separate data for buses with engine problems and body problems
engine_problems = df[df['Problem'] == 'Engine']
body_problems = df[df['Problem'] == 'Body']

# List of SPN columns
spn_columns = ['SPN_247 (Engine Total Hours of Operation)', 'SPN_190 (Engine Speed)',
               'SPN_513 (Actual Engine - Percent Torque)', 'SPN_250 (Engine Total Fuel Used)',
               'SPN_110 (Engine Coolant Temperature)', 'SPN_183(Engine Fuel Rate)',
               'SPN_184 (Engine Instantaneous Fuel Economy)', 'SPN_92\t(Engine Percent Load At Current Speed)']

# Calculate summary statistics for each SPN column for engine problems
engine_summary = engine_problems[spn_columns].describe()

# Calculate summary statistics for each SPN column for body problems
body_summary = body_problems[spn_columns].describe()

# Print the summary statistics for engine problems
print("Summary Statistics for Buses with Engine Problems:")
print(engine_summary)

# Print the summary statistics for body problems
print("\nSummary Statistics for Buses with Body Problems:")
print(body_summary)


Summary Statistics for Buses with Engine Problems:
       SPN_247 (Engine Total Hours of Operation)  SPN_190 (Engine Speed)  \
count                               2.100000e+01               21.000000   
mean                                9.974833e+05              801.428571   
std                                 3.140541e+06               94.406341   
min                                 2.470000e+02              639.000000   
25%                                 1.226700e+04              755.000000   
50%                                 1.553800e+04              801.000000   
75%                                 1.752200e+04              857.000000   
max                                 1.368495e+07             1040.000000   

       SPN_513 (Actual Engine - Percent Torque)  \
count                                 21.000000   
mean                                  61.952381   
std                                   54.725201   
min                                   18.000000   
25%      

In [5]:
df_model = df.drop(['BUS ID','BR_Repair status','Problem'], axis = 1)


# Splitting and Scaling Dataset and Training on Data

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


# Map grades to numeric values (A: 0, B: 1, C: 2)
df_model['Grade'] = df_model['Grade'].map({'A': 0, 'B': 1, 'C': 2})

# Splitting the dataset
dataset_features = df_model.drop(['Grade'], axis=1)
dataset_label = df_model['Grade']

X_train, X_test, Y_train, Y_test = train_test_split(
    dataset_features, dataset_label, test_size=0.2, random_state=1, stratify=dataset_label
)

print("x_train shape: ", X_train.shape)
print("x_test shape: ", X_test.shape)
print("y_train shape: ", Y_train.shape)
print("y_test shape: ", Y_test.shape)
print("Number of classes ", len(np.unique(Y_train)))

# Initialize and train the Gaussian Naive Bayes model
gnb = GaussianNB()
gnb.fit(X_train, Y_train)

# Make predictions on the testing data
Y_pred = gnb.predict(X_test)

# Create a copy of the testing data DataFrame
testing_data = X_test.copy()

# Add the 'BUS ID' and predicted 'Grade' columns
testing_data['BUS ID'] = df.loc[X_test.index, 'BUS ID']  # Assuming 'BUS ID' is a column in df
testing_data['Grade'] = Y_pred


# Calculate and print evaluation metrics
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1 Score: {:.2f}".format(f1))

# Print the confusion matrix
confusion = confusion_matrix(Y_test, Y_pred)
print("Confusion Matrix:")
print(confusion)


x_train shape:  (38, 8)
x_test shape:  (10, 8)
y_train shape:  (38,)
y_test shape:  (10,)
Number of classes  3
Accuracy: 0.70
Precision: 0.61
Recall: 0.70
F1 Score: 0.62
Confusion Matrix:
[[5 0 0]
 [1 2 0]
 [2 0 0]]


C:\Users\shake\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Predicting

In [7]:
filtered_df = pd.read_csv("prediction_data.csv")
Y_pred = gnb.predict(filtered_df.drop(['BUS ID'], axis=1))
result_df = pd.DataFrame({'BUS ID': filtered_df['BUS ID'], 'Grade': Y_pred})
result_df.to_csv('predicted_grades.csv', index=False)